# import libraries

In [2]:
# data manipulation 
import numpy as np
import pandas as pd

# plotting
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

# setting params
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (30, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

sn.set_style('whitegrid')
sn.set_context('talk')

plt.rcParams.update(params)
pd.options.display.max_colwidth = 600

# pandas display data frames as tables
from IPython.display import display, HTML


# load dataset

In [31]:
df = pd.read_csv('/Users/ingluissantana/Desktop/Py/PP1_bike_sharing_MVG/MVG_Rad_Fahrten_2021.csv', delimiter= ';', index_col=0)



In [32]:
df.head().T

Row,1,2,3,4,5
STARTTIME,2021-01-01 01:01,2021-01-01 01:19,2021-01-01 01:48,2021-01-01 01:48,2021-01-01 03:26
ENDTIME,2021-01-01 01:12,2021-01-01 01:59,2021-01-01 02:01,2021-01-01 02:00,2021-01-01 03:39
STARTLAT,48.1258,48.12919,48.08189,48.08189,48.11587
STARTLON,11.64784,11.62583,11.63264,11.63264,11.62543
ENDLAT,48.12948,48.14853,48.07975,48.07975,48.109
ENDLON,11.62539,11.53142,11.61032,11.61032,11.6524
RENTAL_IS_STATION,1,0,1,1,0
RENTAL_STATION_NAME,Kreillerstraße,,Universitätsstraße Neubiberg,Universitätsstraße Neubiberg,
RETURN_IS_STATION,0,0,1,1,0
RETURN_STATION_NAME,,,Fasanenpark Ost Neubiberg,Fasanenpark Ost Neubiberg,


# check for null values